### requests
- 爬虫中一个基于网络请求的模块
- pip install requests
- 作用:模拟浏览器发起请求
- 编码流程:
    - 1.指定url
    - 2.发起请求
    - 3.获取响应数据(爬取到的页面源码数据)
    - 4.持久化存储

In [1]:
import requests

In [6]:
#1 指定url
url = 'https://www.google.com/'
#2 发起请求  get方法的返回值为响应对象
response = requests.get(url = url)
#3 获取响应数据  text是字符串形式的响应数据
page_text = response.text
#4 持久化存储
with open('./google.html','w',encoding='utf-8') as fp:
    fp.write(page_text)

- 实现一个简易的网页采集器
    - 基于谷歌针对指定不同的关键字将其对应的页面数据进行爬取
    - 参数动态化:
        - 如果请求的url携带参数,且我们想要将携带的参数进行动态化操作,那么我们必须:
            - 1.将携带的参数以KV对的方式封装到一个字典中
            - 2.将该字典作用到get方法的params参数中即可
            - 3.需要将原始携带的参数的url中携带的参数删除

In [19]:
keyWord = input('Enter a key word:')

# 携带了请求参数的url,如果想要爬取不同关键字对应的页面,我们需要将url携带的参数进行动态化
# 实现参数动态化:封装一个字典,把我们的参数封装到字典中,再用字典将参数传入get方法
params = {
    'q':keyWord
}

url = 'https://www.google.com/search'
# params参数(字典): 保存请求时url携带的参数

response = requests.get(url = url, params = params)
response.encoding = 'utf-8'  # 如果网页不是utf-8的编码,缺少这一步会出现异常
page_text = response.text

fileName = keyWord + '.html'
with open(fileName,'w',encoding='utf-8') as fp:
    fp.write(page_text)
print(fileName, '爬取完毕')

Enter a key word:泠鸢yousa
泠鸢yousa.html 爬取完毕


- 处理乱码后,页面显示[异常访问请求]导致请求数据的缺失
    - 异常的访问请求
        - 网站后台已经检测出该请求不是通过浏览器发起的请求,而是通过爬虫发起的请求.(不是浏览器发起的请求都是异常请求)
    - 网站的后台是如何知道是不是通过浏览器发起的呢?
        - 是通过判定请求头中的user-agent判定的
- 什么是user-agent
    - 请求载体的身份标识(身份表示可以从抓包工具中获取)
    - 什么是请求载体
        - 浏览器
            - 浏览器的身份标识是统一固定的.
        - 爬虫程序
            - 身份标识是各自不同的.

- 第二种反扒机制:
    - UA检测: 网站后台会检测请求对应的user-agent,以判定当前请求是否异常.
- 反反爬策略:
    - UA伪装: 被作用到了大部分的网站中,日后我们写的爬虫都默认带上UA检测操作.
    - 伪装流程:
        - 从抓包工具中捕获到某一个基于浏览器请求的user-agent的值,将其伪装作用到一个字典中,将盖子点作用到请求方法(get,post)的headers参数中即可.

In [20]:
keyWord = input('Enter a key word:')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36 Edg/89.0.774.63'}
# 携带了请求参数的url,如果想要爬取不同关键字对应的页面,我们需要将url携带的参数进行动态化
# 实现参数动态化:封装一个字典,把我们的参数封装到字典中,再用字典将参数传入get方法
params = {'q':keyWord}

url = 'https://www.google.com/search'
# params参数(字典): 保存请求时url携带的参数

response = requests.get(url = url, params = params, headers = headers)
response.encoding = 'utf-8'  # 如果网页不是utf-8的编码,缺少这一步会出现异常
page_text = response.text

fileName = keyWord + '.html'
with open(fileName,'w',encoding='utf-8') as fp:
    fp.write(page_text)
print(fileName, '爬取完毕')

Enter a key word:阿夸
阿夸.html 爬取完毕


- 爬取豆瓣电影中的电影详情数据
    - url:https://www.douban.com/channel/30168848/?source=search
    - 动态加载数据的捕获
        - 什么叫做动态加载的数据?
            - 我们通过requests模块进行数据爬取无法每次都实现可见即可得.
            - 是因为有些数据是通过非浏览器地址栏中的url请求到的数据,而是其他请求请求到的数据,那么这些通过其他请求来请求到的数据就是动态加载的数据.
    - 如何检测网页中是否存在动态加载数据?
        - 基于抓包工具进行局部搜索.
            - 在当前网页中打开抓包工具,搜索到地址栏的url对应的数据包,在该数据包的response(响应)选项卡中搜索我们想要爬取的数据,如果搜索到了结果则表示数据不是动态加载的,否则表示数据为动态加载的.

In [23]:
url='https://www.douban.com/channel/30168848/?source=search'
response = requests.get(url=url,headers=headers)
response.encoding = 'utf-8'
page_text=response.text
with open('./douban.html','w',encoding='utf-8') as fp:
    fp.write(page_text)